<a href="https://colab.research.google.com/github/JahanviGupta17/Chat-With-Pdfs-RAG-Powered-PDF-Research-Assistant/blob/main/chat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# Colab: Install required packages
!pip install streamlit pyngrok langchain sentence-transformers transformers pinecone pdfplumber langchain-text-splitters langchain-huggingface langchain-pinecone

# Import libraries
import os
import streamlit as st
import pdfplumber
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
import pinecone
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
import torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.6/587.6 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.8/84.8 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 111.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.3/259.3 kB 27.5 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: pinecone-plugin-assistant
    Found existing installation: pinecone-plugin-assistant 3.0.1
    Uninstalling pinecone-plugin-assistant-3.0.1:
      Successfully uninstalled pinecone-plugin-assistant-3.0.1
  Attempting uninstall: pinecone
    Found existing installation: pinecone 8.0.0
    Uninstalling pinecone-8.0.0:
      Successfully uninstalled pinecone-8.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the f

In [3]:
import os
import streamlit as st
import pdfplumber
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_pinecone import Pinecone as PineconeVectorStore
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
import torch
import pinecone


In [9]:
!pip install --upgrade "pinecone-client>=5.2.0"


In [20]:
# ⚠️ Only for testing, don't share this publicly
PINECONE_API_KEY = "pcsk_6Sbvwk_AAz6zejmVbJ8wfXixysVjt1jDPiQ2cL3JAmHND4PjD7R9YGkDogpZPbHB4GNrRz"
PINECONE_ENV = "us-east1-gcp"

print("✅ Pinecone key loaded successfully")


✅ Pinecone key loaded successfully


In [21]:
# Using fast embeddings for large PDFs
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [22]:
# Local LLM using FLAN-T5-small (fast for short summaries)
llm_model_name = "google/flan-t5-small"
device = 0 if torch.cuda.is_available() else -1

tokenizer = AutoTokenizer.from_pretrained(llm_model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(llm_model_name)

llm_pipeline = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    device=device,
    max_length=512
)


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Device set to use cuda:0


In [23]:
def extract_pdf_text(pdf_files):
    text = ""
    for pdf in pdf_files:
        with pdfplumber.open(pdf) as pdf_doc:
            for page in pdf_doc.pages:
                page_text = page.extract_text()
                if page_text:
                    text += page_text + "\n"
    return text


In [24]:
def split_text(text, chunk_size=1000, overlap=200):
    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=overlap)
    return splitter.split_text(text)


In [25]:
def index_documents(chunks):
    # Remove empty chunks
    chunks = [c for c in chunks if c.strip()]

    # Convert to embeddings and upsert
    vectors = [(str(i), embeddings.embed_documents([chunk])[0], {"text": chunk}) for i, chunk in enumerate(chunks)]
    index.upsert(vectors)
    st.success(f"✅ {len(chunks)} chunks indexed into Pinecone")


In [26]:
def retrieve_context(query, top_k=5):
    # Retrieve top_k relevant chunks from Pinecone
    results = index.query(queries=[embeddings.embed_query(query)], top_k=top_k, include_metadata=True)
    docs = [match['metadata']['text'] for match in results['results'][0]['matches']]
    return docs

def summarize_chunks(chunks):
    summaries = []
    for i, chunk in enumerate(chunks):
        prompt = f"Summarize the following text in 1-2 sentences. Cite as [Source {i+1}].\n\n{chunk}\nSummary:"
        response = llm_pipeline(prompt)
        summaries.append(response[0]['generated_text'].strip())
    return " ".join(summaries)


In [27]:
def generate_answer(question):
    chunks = retrieve_context(question, top_k=5)
    if not chunks:
        return "Not enough information"

    context = summarize_chunks(chunks)
    prompt = f"""
You are a research assistant AI.
Answer the question concisely using only the provided context.
Cite sources like [Source X].
If information is missing, say 'Not enough information'.

Context:
{context}

Question:
{question}

Answer:
"""
    response = llm_pipeline(prompt)
    return response[0]['generated_text'].strip()


In [28]:
def main():
    st.set_page_config(page_title="ResearchBot", page_icon="📚", layout="wide")

    st.markdown(
        """
        <style>
        body {background-color:#0d1117; color:#c9d1d9;}
        .stButton>button {background-color:#238636; color:white; border-radius:10px;}
        .stTextInput>div>input {background-color:#161b22; color:white;}
        </style>
        """,
        unsafe_allow_html=True
    )

    st.title("📚 ResearchBot – Research Paper Assistant")

    question = st.text_input("Enter your research question:")
    if question:
        answer = generate_answer(question)
        st.subheader("🤖 Answer")
        st.write(answer)

    with st.sidebar:
        st.header("📁 Upload PDFs")
        pdfs = st.file_uploader("Upload your research papers", type="pdf", accept_multiple_files=True)
        if st.button("Index PDFs"):
            if not pdfs:
                st.warning("Please upload at least one PDF.")
            else:
                text = extract_pdf_text(pdfs)
                chunks = split_text(text)
                index_documents(chunks)

if __name__ == "__main__":
    main()


2026-01-29 21:11:13.283 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-29 21:11:13.288 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-29 21:11:13.995 
  command:

    streamlit run /usr/local/lib/python3.12/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2026-01-29 21:11:13.998 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-29 21:11:14.004 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-29 21:11:14.005 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-29 21:11:14.006 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-29 21:11:14.010 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running 